### OCR

In [14]:
# Importing the libraries
import pandas as pd
import numpy as np
from datetime import datetime

import os

import pytesseract
import PIL.Image
if not hasattr(PIL.Image, 'Resampling'):
    PIL.Image.Resampling = PIL.Image

from PIL import Image

import cv2
import easyocr

from tqdm import tqdm

import matplotlib.pyplot as plt

import re

In [3]:
# image directory
image_dir = '../images/aculei-images/'

In [4]:
# reading the dataframe
df = pd.read_csv('../aculei.csv', index_col=0)

We want to fill the missing data which were not available through metadata using ocr techniques.

The missing data are:

- some **cameras**
- all **temperatures**

So we have to filter the dataframe and try to fill in the missing data for the camera column and then we'll do the same thing for the temparature column (which is not present yet)


In [5]:
df.head()

,image_name,camera,date_time,date,time,moon
0,TF_ACULEI_8040_DSCF0129.jpg,CAM_1,2021-07-22 23:04:07,2021-07-22,23:04:07,Full Moon
1,TF_ACULEI_900_DSCF0756.jpg,CAM_1,2021-05-27 22:01:44,2021-05-27,22:01:44,Full Moon
2,TF_ACULEI_15294_DSCF0133.jpg,NaN,2023-03-30 20:35:58,2023-03-30,20:35:58,First Quarter
3,TF_ACULEI_11374_DSCF0064.jpg,CAM_6,2022-06-09 21:29:59,2022-06-09,21:29:59,Waxing Gibbous
4,TF_ACULEI_4106_DSCF4336.jpg,CAM_1,2021-06-16 23:41:09,2021-06-16,23:41:09,First Quarter


In [6]:
df.shape

(16874, 6)

In [7]:
df.isnull().sum()

image_name       0
camera        4452
date_time      103
date           103
time           103
moon           103
dtype: int64

From the output above we can see we are missing

- 4452 cameras
- 103 dates
- 16874 temperatures

We'll split the jobs to keep it simple

### Cameras

We'll filter the dataset keeping only the empty camera rows (we expect 4452 rows)

In [8]:
df_ocr = df.copy()

In [9]:
df_ocr.shape

(16874, 6)

In [10]:
df_ocr.sample(5)

,image_name,camera,date_time,date,time,moon
13206,TF_ACULEI_3959_DSCF4177.jpg,CAM_1,2021-06-16 00:34:21,2021-06-16,00:34:21,First Quarter
6598,TF_ACULEI_9195_IMAG0071.jpg,CAM_3,2021-09-09 21:57:48,2021-09-09,21:57:48,Waxing Crescent
14555,TF_ACULEI_11065_DSCF0212.jpg,CAM_4,2022-05-01 01:39:15,2022-05-01,01:39:15,New Moon
11237,TF_ACULEI_10240_IMAG0035.jpg,CAM_3,2022-01-05 22:04:40,2022-01-05,22:04:40,Waxing Crescent
7163,TF_ACULEI_3743_DSCF3961.jpg,CAM_1,2021-06-15 23:52:14,2021-06-15,23:52:14,First Quarter


Using [pytesseract](https://pypi.org/project/pytesseract/) we can extract text from the images. We decided to crop the images to achieve better results.
The output string from pytesseract is not tokenized or anything so we used RegExp to extract relevant information from the whole string


In [25]:
def parse_date(date_string):
        for format in ["%Y/%m/%d", "%d/%m/%Y"]:
            try:
                return datetime.strptime(date_string, format)
            except ValueError:
                pass
        return None

In [33]:
for index, row in tqdm(df_ocr.iterrows(), total=len(df_ocr)):
    image_path = os.path.join(image_dir, row['image_name'])
    if os.path.isdir(image_path) and image_path.startswith("."):
        continue 
    
    image = Image.open(image_path)
    
    # get image dimensions
    width, height = image.size
    
    # define the coordinates for cropping
    left = 0
    upper = 0
    right = width
    lower = int(height * 17 / 18)  # keep only the bottom 1/3 of the image
    
    # crop the image
    cropped_image = image.crop((left, lower, right, height))
    
    text = pytesseract.image_to_string(cropped_image, config='--psm 3')

    celsius_pattern = r'\b\d+\s*[°]\s*C\b'
    cam_pattern = r'\bCA\S*'
    date_pattern = r'\b\d{4}[-/]\d{2}[-/]\d{2}\b|\b\d{2}[-/]\d{2}[-/]\d{4}\b'    
    time_pattern = r'\b\d{1,2}:\d{2}:\d{2}\b'

    celsius_matches = re.findall(celsius_pattern, text)
    cam_matches = re.findall(cam_pattern, text)
    date_matches = re.findall(date_pattern, text)
    time_matches = re.findall(time_pattern, text)
    
    # saving results
    row["temp"] = celsius_matches[0] if celsius_matches else None
    row["camera"] = cam_matches[0] if cam_matches else None
    
    # preprocessing the date
    if date_matches:
        parsed_date = parse_date(date_matches[0])
        if parsed_date is not None:
            formatted_date = parsed_date.strftime("%Y-%m-%d")
            row["date"] = formatted_date
   
    row["time"] = time_matches[0] if time_matches else None
   
    if time_matches and formatted_date:
        row["datetime"] = formatted_date + " " + time_matches[0] 

  1%|          | 176/16874 [01:29<2:20:54,  1.97it/s]


KeyboardInterrupt: 

In [ ]:
df_camera.head(5)

It seems that the ocr recognized a lot of cameras but some of them are not well formatted (e.g CAME), so we have to understand what they stands for and substitute with the correct camera if possible. Otherwise we can simply drop them.

In [ ]:
df_camera['camera'].value_counts()

In [ ]:
df_camera.isnull().sum()

We reached only 537 falsy cameras but we decided to drop the ones not well formatted

In [ ]:
%%capture

# drop the camera from ocr not well formatted
valid_cameras = [f'CAM{i}' for i in range(1, 8)]
df_camera['camera'] = df_camera['camera'].apply(lambda x: x if x in valid_cameras else None)

In [ ]:
df_camera.isnull().sum()

In [ ]:
df_camera['camera'].value_counts()

We want to format the camera recognized with the ocr (from CAM7 to CAM_7)

In [ ]:
fail here

So we lost only **2395** cameras using OCR for camera recognition

Now let's check some results

In [ ]:
import matplotlib.pyplot as plt
import random

image_folder_path = '../images/aculei-images/'

# extract four random images from the dataset
valid_rows = df_camera.dropna(subset=['camera'])
six_random_rows = valid_rows.sample(n=6, random_state=random.seed())
image_folder_path = '../images/aculei-images/'

fig, ((ax1, ax2, ax3), (ax4, ax5, ax6)) = plt.subplots(2, 3, figsize=(16, 9))
fig.suptitle('OCR results', fontsize=15)

axes = [ax1, ax2, ax3, ax4, ax5, ax6]

for i, ax in enumerate(axes):
    image_path = os.path.join(image_folder_path, six_random_rows.iloc[i]['image_name'])
    image = Image.open(image_path)
    ax.imshow(image, aspect='auto')
    ax.axis('off')

    camera_info = six_random_rows.iloc[i]['camera']
    ax.set_title(f"Camera recognized by ocr: {camera_info}", fontsize=10)

plt.tight_layout()
plt.show()

Now we can merge results in the original dataframe and save it! we didn't compute the hash so we use the image name as merge key for now

In [ ]:
# checking if image name is a safe key
print(len(df['image_name']))
print(df.shape[0])

In [ ]:
merged_camera = df.merge(df_camera[['image_name', 'camera']], on='image_name', how='left', suffixes=('', '_camera'))

# Fill missing 'camera' values in 'df' with values from 'df_camera'
merged_camera['camera'] = merged_camera['camera'].combine_first(merged_camera['camera_camera'])

# Drop the extra 'camera_camera' column
merged_camera.drop('camera_camera', axis=1, inplace=True)  


In [ ]:
merged_camera.isnull().sum()

In [ ]:
# save it!
merged_camera.to_csv('../aculei.csv')